---
title: "Procesamiento de Señales e Imagenes"
description: "PSIM -- 101849"
subtitle: "Ingeniería Biomédica"
lang: es
author: "Ph.D. Pablo Eduardo Caicedo Rodríguez"
date: last-modified
format:
  revealjs: 
    code-tools: true
    code-overflow: wrap
    code-line-numbers: true
    code-copy: true
    fig-align: center
    self-contained: true
    theme: 
      - simple
      - ../../recursos/estilos/metropolis.scss
    slide-number: true
    preview-links: auto
    logo: ../../recursos/imagenes/generales/Escuela_Rosario_logo.png
    css: ../../recursos/estilos/styles_pres.scss
    footer: <https://pablocaicedor.github.io/>
    transition: fade
    progress: true
    scrollable: true
    mainfont: "Fira Code"
---

# Un recorrido narrado por señales e imágenes

Hoy nos reunimos para observar cómo los datos del cuerpo cuentan historias. Pensaremos en una señal como una narración que avanza segundo a segundo —como una línea que respira— y en una imagen como una narración detenida en el espacio, organizada en diminutos cuadros llamados píxeles. No necesitaremos fórmulas: bastará con la curiosidad y con la disposición para experimentar. Al final habrás creado tus propias figuras y podrás explicar, con tus palabras, qué cambió cuando el ruido apareció o cuando un filtro suavizó la escena.

---

# La primera escena: escuchar el corazón

Imagina que acercas un micrófono al pecho. Lo que el sensor “oye” se convierte en una secuencia de números tomada muchas veces por segundo. A ese ritmo de toma lo llamamos *muestreo*, y su papel es parecido al de una cámara lenta: si tomamos muy pocos “fotogramas”, la historia pierde detalle; si tomamos suficientes, podemos seguir el ritmo del latido. Cada número ocupa un espacio en el computador; ese espacio define qué tan finamente distinguimos entre “más fuerte” y “más suave”. Esta finura se asocia con *profundidad de bits*. Sin ecuaciones, basta con la idea: más muestras y pasos más finos suelen permitir descripciones más nítidas, aunque siempre tendremos que escoger con cuidado.

---

# Una prueba con Python: sonido ideal, ruido y suavizado

Enseguida construiremos una señal sencilla que late. Luego le añadiremos ruido —pequeñas sacudidas aleatorias— y por último la suavizaremos con un promedio deslizante. Observa cómo la claridad mejora, pero también cómo algunos detalles se aplanan.

In [ ]:
#| label: setup
#| echo: false
import numpy as np, matplotlib.pyplot as plt, seaborn as sns
rng = np.random.default_rng(42)
sns.set()

In [ ]:
#| label: narrativa-senal
#| fig-cap: 'Una historia en el tiempo: señal ideal, señal con ruido y señal suavizada.'
fs = 200               # 'muestras por segundo' para narrar con fluidez
t  = np.arange(0, 5, 1/fs)
x  = 0.8*np.sin(2*np.pi*1.2*t) + 0.3*np.sin(2*np.pi*3.5*t)  # un 'latido' estilizado
ruido = 0.4*rng.normal(size=t.size)
y  = x + ruido

# Suavizado por promedio (ventana impar)
k = 9
kernel = np.ones(k)/k
y_suave = np.convolve(y, kernel, mode='same')

plt.figure(figsize=(9,3.2))
plt.plot(t, x, lw=1.5, label="Relato ideal")
plt.plot(t, y, alpha=.6, label="Relato con ruido")
plt.plot(t, y_suave, lw=2, label="Relato suavizado")
plt.xlabel("Tiempo [s]"); plt.ylabel("Amplitud (adim.)")
plt.legend(); plt.tight_layout(); plt.show()

---

# La segunda escena: mirar el espacio

Una imagen no avanza; se despliega ante nosotros como un mosaico. Cada píxel conserva un valor de brillo o color. Cuantos más píxeles haya en un área dada, más fino será el mosaico y, por tanto, más detalle veremos. Cuando una imagen se vuelve borrosa —por movimiento, por desenfoque o por compresión— las transiciones se suavizan y los bordes pierden definición. Podemos intentar recuperarlos detectando dónde cambian rápido los valores: allí suelen esconderse las líneas y contornos que la mirada reconoce.

---

# Un experimento visual: del patrón limpio al borde

Ahora generaremos una imagen sintética. Primero veremos su versión “limpia”; luego la agitaremos con ruido; después la desenfocaremos con un pequeño promedio; por último buscaremos bordes con un operador sencillo. Observa cómo el desenfoque ayuda a reducir el grano pero, si se exagera, disuelve contornos que tal vez queríamos conservar.

In [ ]:
#| label: narrativa-imagen
#| fig-cap: 'Una historia en el espacio: patrón base, ruido, desenfoque y bordes.'
def pad_reflect(A, p):
    return np.pad(A, ((p,p),(p,p)), mode='reflect')

def conv2(A, K):
    p = K.shape[0]//2
    Ap = pad_reflect(A, p)
    out = np.zeros_like(A, dtype=float)
    for i in range(out.shape[0]):
        for j in range(out.shape[1]):
            region = Ap[i:i+K.shape[0], j:j+K.shape[1]]
            out[i,j] = np.sum(region * K)
    return out

# Patrón de prueba: degradé + tableros
n = 180
x = np.linspace(0,1,n)
grad = np.tile(x, (n,1))
checker = (((np.indices((n,n)).sum(axis=0))%30) < 15).astype(float)
img = 0.6*grad + 0.4*checker

# Ruido y desenfoque
img_r = img + 0.12*rng.normal(size=img.shape)
K = np.ones((5,5))/25
img_blur = conv2(img_r, K)

# Bordes (Sobel)
Sx = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
Sy = Sx.T
gx, gy = conv2(img_blur, Sx), conv2(img_blur, Sy)
edges = np.hypot(gx, gy)

fig, axs = plt.subplots(1,4, figsize=(12,3))
for a,im,tit in zip(axs, [img, img_r, img_blur, edges], 
                    ["Base", "Con ruido", "Desenfoque", "Bordes"]):
    a.imshow(im, cmap='gray'); a.set_title(tit); a.axis('off')
plt.tight_layout(); plt.show()

---

# Actividad guiada I: cuenta tu propia señal

Te propongo crear tu propia narración en el tiempo. Modifica las frecuencias para imitar un ritmo distinto; aumenta o reduce el ruido para ver en qué momento la historia deja de ser legible; ajusta la ventana del suavizado y describe, con dos o tres frases, el compromiso entre claridad y detalle. No hay respuestas únicas: nos interesa cómo justificas tus decisiones.

In [ ]:
#| label: practica-narrativa-senal
#| fig-cap: 'Cuenta tu señal: ajusta el ritmo, el ruido y el suavizado y explica tu elección.'
fs = 300
t  = np.arange(0, 6, 1/fs)

f1, f2 = 1.0, 2.2           # experimenta con estos valores
x  = 1.0*np.sin(2*np.pi*f1*t) + 0.4*np.sin(2*np.pi*f2*t)
ruido = 0.30*rng.normal(size=t.size)   # modifícalo para tensar la historia
y  = x + ruido

k = 11                       # el tamaño de la 'ventana' moldea el tono
kernel = np.ones(k)/k
y_suave = np.convolve(y, kernel, mode='same')

fig, ax = plt.subplots(2, 1, figsize=(9,6), sharex=True)
ax[0].plot(t, x, lw=1.5, label="Ideal"); ax[0].plot(t, y, alpha=.6, label="Con ruido")
ax[0].set_title("Tu relato con interferencias")
ax[0].legend()

ax[1].plot(t, y, alpha=.5, label="Ruidosa"); ax[1].plot(t, y_suave, lw=2, label="Suavizada")
ax[1].set_title(f"Tu herramienta de calma (ventana={k})")
ax[1].set_xlabel("Tiempo [s]"); 
for a in ax: a.set_ylabel("Amplitud")
ax[1].legend(); plt.tight_layout(); plt.show()

---

# Actividad guiada II: esculpe tu imagen

Ahora construye una escena propia: un degradé, un círculo, un motivo que te guste. Aumenta el ruido hasta que la textura se vuelva áspera y luego atenúala con un desenfoque moderado. Observa cuándo los contornos emergen con el detector de bordes y cuándo, en cambio, se diluyen. Concluye con un pie de foto que explique, sin jerga, qué aprendiste sobre el equilibrio entre nitidez y suavidad.

In [ ]:
#| label: practica-narrativa-imagen
#| fig-cap: 'Esculpe la escena: patrón, ruido, desenfoque, bordes y tu interpretación final.'
n = 200
x = np.linspace(0,1,n)
grad = np.tile(x**0.6, (n,1))
yy, xx = np.indices((n,n))
circulo = ((xx-n/2)**2 + (yy-n/2)**2 < (n/4)**2).astype(float)
img = 0.7*grad + 0.3*circulo

ruido_sigma = 0.18
img_r = img + ruido_sigma*rng.normal(size=img.shape)

k = 7
K = np.ones((k,k))/k**2
img_blur = conv2(img_r, K)

Sx = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
Sy = Sx.T
gx, gy = conv2(img_blur, Sx), conv2(img_blur, Sy)
edges = np.hypot(gx, gy)

fig, axs = plt.subplots(1,4, figsize=(12,3))
for a,im,tit in zip(axs, [img, img_r, img_blur, edges], 
                    ["Patrón propio", f"Ruido σ={ruido_sigma}", f"Desenfoque k={k}", "Bordes"]):
    a.imshow(im, cmap='gray'); a.set_title(tit); a.axis('off')
plt.tight_layout(); plt.show()

---

# Ética y cuidado en cada decisión

En esta sesión usamos datos sintéticos, precisamente para practicar sin arriesgar la privacidad de nadie. En proyectos reales, las señales y las imágenes pertenecen a personas, y tratarlas con respeto implica proteger identidades, explicar el propósito de uso y documentar las decisiones de procesamiento. La narrativa técnica siempre convive con una narrativa humana: quiénes participaron, qué beneficios esperan y cómo reducimos riesgos.

---

# Cierre: lo que la historia nos deja

Has visto que una señal puede volverse ruidosa y que un filtro, al calmarla, también puede borrar matices. Has visto que una imagen puede ganar suavidad y perder aristas, o recuperarlas con un detector de bordes. La idea central es sencilla: toda herramienta implica un intercambio. Nuestro trabajo consiste en elegir con sentido, observar con cuidado y explicar con palabras claras cuál fue el camino y por qué.

::: notes
Para el docente: invita a que cada equipo muestre **un** gráfico y lo resuma en dos frases. Pide que digan explícitamente qué parámetro movieron, qué efecto observaron y qué decisión tomarían si el objetivo fuera ver el latido con nitidez o delinear un borde en una radiografía.
:::